In [41]:
# conda activate movie-recommender          # activate environment in terminal         


In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
chunksize=100000
import seaborn as sns
import matplotlib.pyplot as plt


In [43]:


# ratings
movies = pd.DataFrame()  # initialize an empty dataframe
for index, chunk in enumerate(pd.read_csv('../ml-20m/movies.csv',
                         dtype={'movieId': str, 'title': str, "genres": str},
                         chunksize=chunksize,
                         )):
    # Perform operations on the chunk
    # processed_chunk = chunk[chunk['column_name'] > 0]  # Example operation
    movies = pd.concat([movies, chunk])
    print(f"Chunk {index} loaded")
print("movies dataset loaded")

Chunk 0 loaded
movies dataset loaded


In [44]:
movies.nunique()

movieId    27278
title      27262
genres      1342
dtype: int64

In [45]:
movies["genres"].unique()

array(['Adventure|Animation|Children|Comedy|Fantasy',
       'Adventure|Children|Fantasy', 'Comedy|Romance', ...,
       'Action|Adventure|Animation|Fantasy|Horror',
       'Animation|Children|Comedy|Fantasy|Sci-Fi',
       'Animation|Children|Comedy|Western'], dtype=object)

In [50]:
# Splitting the 'Genres' column into multiple columns
split_genres = movies['genres'].str.split('|', expand=True)

# Filling NaN values with a specific value, e.g., 'None'
split_genres = split_genres.fillna('None')
genre_list_size=len(split_genres.columns)
genre_list = [f'genre_{i}' for i in range(1, genre_list_size+1)]
split_genres.columns = genre_list 
# Adding the split columns back to the original DataFrame
# movies = movies.join(split_genres)
movies_split_genres = pd.merge(movies, split_genres, left_index=True, right_index=True)
movies_split_genres[genre_list].value_counts()

genre_1      genre_2    genre_3    genre_4  genre_5  genre_6  genre_7  genre_8  genre_9  genre_10
Drama        None       None       None     None     None     None     None     None     None        4520
Comedy       None       None       None     None     None     None     None     None     None        2294
Documentary  None       None       None     None     None     None     None     None     None        1942
Comedy       Drama      None       None     None     None     None     None     None     None        1264
Drama        Romance    None       None     None     None     None     None     None     None        1075
                                                                                                     ... 
Animation    Comedy     Fantasy    Musical  Romance  None     None     None     None     None           1
                        Horror     IMAX     None     None     None     None     None     None           1
                                   Musical  None     N

next i want ot stack the genres on top of eachother so that there will be one column of genres to each movie